In [22]:
import os
import numpy as np
import pandas as pd


from bs4 import BeautifulSoup
import re
from collections import OrderedDict

pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', 2000)

In [2]:
law_sum_dir = 'law_summaries/'
law_summaries_html = os.listdir(law_sum_dir)
law_summaries_html.sort()
law_summaries_html.pop(0)

'.DS_Store'

In [3]:
law_list = []

for ix, fname in enumerate(law_summaries_html):
    
    html_f = open(law_sum_dir + fname,'r')
    page = html_f.read()
    summary_soup = BeautifulSoup(page,"lxml")
    
    
    temp_law_dict = dict()

    # Get Bill Full Nama and title
    title = summary_soup.find('h1',class_ = 'legDetail').text.strip().replace('\n',' ')
    bill_num = title.split('-')[0]


    # Find Congress and Law
    
    summary_table = summary_soup.find('table')
    
    
    for jx, td in enumerate(summary_table.find_all('td')):
            if 'Became Public Law' in td.text:
                    #law_num = td.text.split('Became Public Law No: ')[1].split('-')[1].split('.')[0]
                    temp_split = td.text.split('Became Public Law No: ')[1]
                    temp_split = re.split(r'[\-]|[\.]',temp_split)
                    congress = temp_split[0]
                    law = temp_split[1]


    ths = summary_table.find_all('th')                
    # Find Sponsor and Committee
    
    committee_found = 0 
    for th in ths:
        if 'Sponsor' in th.text:
            
            try:
                sponsor = th.next_sibling.next_sibling.find('a').text
            except AttributeError:
                sponsor = 'None'
                sponsor_party = 'None'
                sponsor_state = 'None'
            else:
                sponsor = sponsor.replace('\n','').strip()
                sponsor_info = sponsor.split('[', 1)[1].split(']')[0]
                sponsor_info = sponsor_info.split('-')
                sponsor_party = sponsor_info[0]
                sponsor_state = sponsor_info[1]

        if ('Committee' in th.text) and ('Report' not in th.text):
            committee_found = 1 
            committee = th.next_sibling.next_sibling.text
            committee = committee.replace('\n','').strip()
    
    if committee_found == 0:
        committee = 'none'

    temp_law_dict['id'] = fname.replace('.html','')
    temp_law_dict['congress'] = congress
    temp_law_dict['law'] = law
    temp_law_dict['title'] = title
    temp_law_dict['bill_num'] = bill_num
    temp_law_dict['sponsor'] = sponsor
    temp_law_dict['sponsor_party'] = sponsor_party
    temp_law_dict['sponsor_state'] = sponsor_state
    temp_law_dict['committee'] = committee
    temp_law_dict['url'] = summary_soup.find('a',text=re.compile('Text'))['href']

    law_list.append(temp_law_dict)
    
    if ix % 100 == 0:
        print(ix)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [5]:
df = pd.DataFrame(law_list)

In [24]:
df.loc[np.random.choice(np.array(range(0,np.shape(law_list)[0])),size = 5),['title','url']]

,title,url
3824,"H.R.2291 - To designate the facility of the United States Postal Service located at 450 Lexington Avenue in New York, New York, as the ""Vincent R. Sombrotto Post Office"". 113th Congress (2013-2014)",https://www.congress.gov/bill/113th-congress/house-bill/2291/text
892,H.R.3544 - Pope John Paul II Congressional Gold Medal Act 106th Congress (1999-2000),https://www.congress.gov/bill/106th-congress/house-bill/3544/text
451,H.R.1273 - National Science Foundation Authorization Act of 1998 105th Congress (1997-1998),https://www.congress.gov/bill/105th-congress/house-bill/1273/text
3106,"H.R.2587 - A Bill to designate the facility of the United States Postal Service located at 555 South 3rd Street Lobby in Memphis, Tennessee, as the ""Kenneth T. Whalum, Sr. Post Office Building"". 110th Congress (2007-2008)",https://www.congress.gov/bill/110th-congress/house-bill/2587/text
3010,H.R.6353 - Ryan Haight Online Pharmacy Consumer Protection Act of 2008 110th Congress (2007-2008),https://www.congress.gov/bill/110th-congress/house-bill/6353/text


In [25]:
df.to_csv('laws_summary.csv',index = False)